In [ ]:
import cv2 as cv
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# ─ CONFIG ─────────────────────────────────────────
'''change the FOLDER NAME AND THE IMAGE NAME'''

BASE_DATE  = "16_7_2025"
FOLDER_NAME = "sample_ycbcr"
IMAGE_NAME = "DJI_0384_zoomed_2.jpg"
BASE_PATH  = Path(f"/Users/at/Orbit_Red_Blob/Data/UKE_Plot_14_DD_{BASE_DATE}/{FOLDER_NAME}")

img_bgr    = cv.imread(str(BASE_PATH / IMAGE_NAME))
if img_bgr is None:
    raise FileNotFoundError(f"Couldn’t load image: {BASE_PATH/IMAGE_NAME}")

# 1) Convert to YCrCb
img_ycrcb = cv.cvtColor(img_bgr, cv.COLOR_BGR2YCrCb)
Y, Cr, Cb = cv.split(img_ycrcb)

c_min, c_max = Cr.min(), Cr.max()
c_5,  c_95   = np.percentile(Cr, 5), np.percentile(Cr, 95)
print("Cr       min/max:", c_min, c_max)
print("Cr   5th/95th pct:", c_5,  c_95)

# 1b) Histogram
plt.hist(Cr.ravel(), bins=256)
plt.title("Cr distribution")
plt.xlabel("Cr value (0–255)")
plt.show()

# 2) Inspect the Cr-channel range
print("Cr min/max:", Cr.min(), Cr.max())
p90 = np.percentile(Cr, 90)
print("Cr 90th percentile:", p90)

# 3) Threshold Cr to pick out the reddest pixels
#    Tweak these bounds based on your printout above
lower_cr = 135     # e.g. 140–160 often works for strong reds
upper_cr = 145
lower_y = 0
upper_y = 50

# lower_cr = 137     # e.g. 140–160 often works for strong reds
# upper_cr = 160
# lower_y = 0
# upper_y = 60


mask = cv.inRange(Cr, lower_cr, upper_cr) & cv.inRange(Y, lower_y, upper_y)

# 4) Optional: clean up noise
mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((3,3),np.uint8))
mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, np.ones((5,5),np.uint8))

# 5) Highlight in the original image
highlight = img_bgr.copy()
highlight[mask > 0] = (0,0,255)   # draw those pixels pure red

# 6) Save out
cv.imwrite(str(BASE_PATH / "output_YCrCb_thresholded.jpg"), highlight)
